In [7]:
import numpy as np
import random
import decimal
import math
from tqdm.notebook import trange

In [8]:
BATCH_SIZE = 16
EPOCHS = 120

In [9]:
def devisionData(size):
    xdata = []
    ydata = []
    for i in range(int(size/BATCH_SIZE)):
        xbatch = []
        ybatch = []
        for j in range(BATCH_SIZE):
            i1, i2 = float(decimal.Decimal(random.randrange(100, 2000))/100), float(decimal.Decimal(random.randrange(100, 2000))/100)
            y = i1 / i2 / 100
            xbatch.append([i1, i2])
            ybatch.append([y])
        xdata.append(np.array(xbatch))
        ydata.append(np.array(ybatch))
    return list(zip(xdata, ydata))

In [10]:
class Model:
    def __init__(self):
        self.lr = 0.00001
        self.W1 = np.random.rand(2, 2) - 0.5
        self.b1 = np.random.rand(2, 1) - 0.5
        self.W2 = np.random.rand(1, 2) - 0.5
        self.b2 = np.random.rand(1, 1) - 0.5
        # self.W1 = np.array([[0.000001, 0.0], [0.0, 0.000001]]) # np.random.randn(2, 2)
        # self.b1 = np.array([[-0.000001], [-0.000001]]) # np.random.randn(2, 1)
        # self.W2 = np.array([[33.3333, -33.3333]]) # np.random.randn(2, 1)
        # self.b2 = np.array([[-3.912023]]) # np.random.randn(1, 1)

    def forward(self, x, predict=True):
        Z1 = self.W1.dot(x) + self.b1
        A1 = self.customActivation(Z1)
        Z2 = self.W2.dot(A1) + self.b2
        A2 = self.customActivation(Z2)
        if predict:
            return A2[0] * 100
        return Z1, A1, Z2, A2

    def backward(self, Z1, A1, Z2, A2, x, y):
        dZ2 = A2 - y # loss
        dW2 = 1/BATCH_SIZE * dZ2.dot(A1.T)
        db2 = 1/BATCH_SIZE * np.sum(dZ2, axis=1, keepdims=True)
        dA1 = self.W2.T.dot(dZ2)
        dZ1 = dA1 * self.customActivationDeriv(Z1)
        dW1 = 1/BATCH_SIZE * dZ1.dot(x.T)
        db1 = 1/BATCH_SIZE * np.sum(dZ1, axis=1, keepdims=True)
        return dW1, db1, dW2, db2, dZ2

    def train(self, train_data, verbose=False):
        for batchIdx, (x, y) in enumerate(train_data):
            x, y = x.T, y.T # Transpose | (batch, input) -> (input, batch)
            Z1, A1, Z2, A2 = self.forward(x, predict=False)
            dW1, db1, dW2, db2, loss = self.backward(Z1, A1, Z2, A2, x, y)
            self.W1 -= self.lr * dW1
            self.b1 -= self.lr * db1
            self.W2 -= self.lr * dW2
            self.b2 -= self.lr * db2
            if verbose and batchIdx % 1000 == 0:
                verbose.set_description(f"Loss: {loss.mean()}")

    def customActivation(self, x):
        oldShape = x.shape
        x = x.reshape(-1)
        for i in range(x.size):
            if x[i] <= 0:
                x[i] = 1.359140915 * math.exp(x[i] - 1)
            elif x[i] > 15:
                x[i] = 1-1/((501379254*x[i])/4596191-280671887/200000) # 1 - 1/(109.0858178 * x[i] - 1403.359435)
            else:
                x[i] = 0.03 * math.log(1000000 * x[i] + 1) + 0.5
        x = x.reshape(oldShape)
        return x

    def customActivationDeriv(self, x):
        oldShape = x.shape
        x = x.reshape(-1)
        for i in range(x.size):
            if x[i] <= 0:
                x[i] = 1.359140915 * math.exp(x[i] - 1)
            elif x[i] > 15:
                x[i] = 92177392592860560000000000/(100275850800000*x[i]-1290021600982417)**2 # 0.00916709/(-12.8647 + x[i])**2
            else:
                x[i] = 30000/(1000000*x[i]+1) # 0.03/(10**-6 + x[i])
        x = x.reshape(oldShape)
        return x

In [11]:
model = Model()

test = np.array([[2], [1]])
print(model.forward(test))
print(model.W1, model.b1, model.W2, model.b2)

[[ 0.29725187 -0.25043917]
 [ 0.31580106  0.17827682]] [[ 0.13443759]
 [-0.30983841]] [[0.33224926 0.27796753]] [[-0.2204177]]
[88.06992785]


In [12]:
train_data = devisionData(128000)
t = trange(EPOCHS)
for epoch in t:
    model.train(train_data, verbose=t)

  0%|          | 0/120 [00:00<?, ?it/s]

ValueError: shapes (2,16) and (2,16) not aligned: 16 (dim 1) != 2 (dim 0)

In [ ]:
print(model.forward(test))
print(model.W1, model.b1, model.W2, model.b2)

[4.51335228]
[[-0.10355816  0.39695988]
 [-0.15737157  0.38222007]] [[0.2624378 ]
 [0.10719376]] [[-0.8346141  -0.77616112]] [[-0.99247994]]
